In [16]:
from pyspark.sql import SparkSession

In [17]:
spark = SparkSession.builder.appName("MLlib").getOrCreate()

In [18]:
training = spark.read.csv("test1.csv", header=True, inferSchema=True)

In [19]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [20]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [21]:
output = featureAssembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [22]:
finalized_data = output.select("Independent Features", "Salary")
finalized_data.show()
# So we have features and target column

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [23]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Independent Features", labelCol="Salary")
model = regressor.fit(train_data)


In [24]:
model.coefficients

DenseVector([-64.8464, 1584.7554])

In [25]:
model.intercept

15414.10693970376

In [26]:
# Prediction
pred_results = model.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [24.0,3.0]| 20000|18612.05915813422|
+--------------------+------+-----------------+



In [27]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1387.9408418657804, 1926379.7805190913)